# ML Experiments using different bin sizes & widths based on outputs from supervised discretization usinf DT discretizer
## Dataset: Satimage

by: Malina & Sam , 03.07.2022

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, recall_score, precision_score
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
import six
import sys
sys.modules['sklearn.externals.six'] = six
import id3
from id3 import Id3Estimator
from id3 import export_graphviz
import wittgenstein as lw
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from pandas import read_csv
from pandas import set_option
import numpy as np
from numpy import arange
## EDA
from collections import Counter

# Knn-VDM 3
from vdm3 import ValueDifferenceMetric
from sklearn.neighbors import KNeighborsClassifier

# Pre-processing
from sklearn.preprocessing import OrdinalEncoder
# Cross validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score # 1 metric
from sklearn.model_selection import cross_validate # more than 1 metric
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [2]:
#read in data from ChiMerge discretization
satimage6 = pd.read_csv('DT_small_discretized_satimage.csv')
satimage8 = pd.read_csv('DT_medium_discretized_satimage.csv')
satimage10 = pd.read_csv('DT_large_discretized_satimage.csv')
satimage15 = pd.read_csv('DT_verylarge_discretized_satimage.csv')

In [3]:
# satimage6 = satimage6.head(500)
# satimage8 = satimage8.head(500)
# satimage10 = satimage10.head(500)
# satimage15 = satimage15.head(500)

In [4]:
satimage6

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A28,A29,A30,A31,A32,A33,A34,A35,A36,class
0,0,0,2,2,1,1,3,2,1,1,...,1,0,0,3,2,2,2,3,2,2
1,0,0,2,2,1,0,3,2,1,0,...,1,0,0,3,2,0,0,3,2,2
2,2,2,1,2,2,2,0,2,2,2,...,2,2,2,0,1,2,2,1,2,0
3,2,1,2,2,2,2,3,2,2,2,...,1,2,0,3,2,2,1,3,2,0
4,2,2,0,0,2,2,0,1,2,2,...,0,2,2,0,0,2,2,0,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6430,2,2,0,0,2,2,0,0,2,2,...,1,2,2,2,1,2,2,2,0,5
6431,1,1,2,2,2,2,2,2,2,2,...,1,1,2,2,2,0,1,3,2,3
6432,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,1
6433,2,2,0,0,2,2,0,0,2,2,...,0,2,2,0,0,2,2,0,1,4


## Interval frequencies

In [5]:
num_list6 = satimage6.columns.drop('class')
num_list8 = satimage8.columns.drop('class')
num_list10 = satimage10.columns.drop('class')
num_list15 = satimage15.columns.drop('class')

In [6]:
print('Interval frequency for 6 Intervals')
for i in num_list6:
    print('Interval for ' + i)
    print(Counter(satimage6[i]))

Interval frequency for 6 Intervals
Interval for A1
Counter({2: 4224, 0: 1048, 1: 617, 3: 546})
Interval for A2
Counter({2: 3210, 0: 1893, 1: 773, 3: 559})
Interval for A3
Counter({2: 3159, 0: 1624, 1: 829, 3: 823})
Interval for A4
Counter({2: 3545, 0: 1295, 1: 1047, 3: 548})
Interval for A5
Counter({2: 4408, 0: 1023, 3: 555, 1: 449})
Interval for A6
Counter({2: 3380, 0: 1855, 1: 629, 3: 571})
Interval for A7
Counter({3: 3222, 0: 1654, 2: 966, 1: 593})
Interval for A8
Counter({2: 3658, 0: 1253, 1: 974, 3: 550})
Interval for A9
Counter({2: 4436, 0: 982, 3: 564, 1: 453})
Interval for A10
Counter({2: 3329, 0: 1732, 1: 775, 3: 599})
Interval for A11
Counter({3: 3183, 0: 1538, 2: 1017, 1: 697})
Interval for A12
Counter({2: 3753, 0: 1262, 1: 852, 3: 568})
Interval for A13
Counter({2: 4393, 0: 1032, 3: 556, 1: 454})
Interval for A14
Counter({2: 3238, 0: 2227, 3: 564, 1: 406})
Interval for A15
Counter({2: 2933, 0: 1739, 3: 1047, 1: 716})
Interval for A16
Counter({2: 3826, 0: 1235, 1: 821, 3: 55

In [7]:
print('Frequency for iris with 8 Intervals')
for i in num_list8:
    print('Interval for ' + i)
    print(Counter(satimage8[i]))

Frequency for iris with 8 Intervals
Interval for A1
Counter({4: 3071, 5: 1153, 1: 934, 7: 484, 2: 461, 3: 156, 0: 114, 6: 62})
Interval for A2
Counter({4: 2621, 1: 1806, 5: 589, 7: 485, 2: 424, 3: 349, 0: 87, 6: 74})
Interval for A3
Counter({5: 2419, 1: 1492, 4: 740, 6: 727, 3: 557, 2: 272, 0: 132, 7: 96})
Interval for A4
Counter({3: 2862, 2: 1221, 4: 728, 5: 683, 7: 500, 1: 319, 0: 74, 6: 48})
Interval for A5
Counter({3: 3227, 5: 1181, 1: 897, 7: 452, 2: 313, 4: 136, 0: 126, 6: 103})
Interval for A6
Counter({3: 2866, 1: 1775, 7: 528, 5: 514, 2: 428, 4: 201, 0: 80, 6: 43})
Interval for A7
Counter({6: 2406, 1: 1519, 7: 816, 4: 519, 5: 447, 2: 386, 3: 207, 0: 135})
Interval for A8
Counter({3: 2977, 2: 1178, 5: 681, 4: 579, 7: 503, 1: 395, 0: 75, 6: 47})
Interval for A9
Counter({2: 3250, 5: 1186, 0: 786, 7: 499, 1: 321, 3: 196, 4: 132, 6: 65})
Interval for A10
Counter({3: 2823, 1: 1664, 7: 544, 5: 506, 4: 503, 2: 272, 0: 68, 6: 55})
Interval for A11
Counter({6: 2398, 1: 1426, 7: 785, 5: 6

In [8]:
print('Frequency for iris with 10 Intervals')
for i in num_list10:
    print('Interval for ' + i)
    print(Counter(satimage10[i]))

Frequency for iris with 10 Intervals
Interval for A1
Counter({4: 2245, 7: 994, 6: 826, 1: 802, 11: 385, 3: 323, 0: 246, 8: 159, 5: 156, 2: 138, 10: 99, 9: 62})
Interval for A2
Counter({5: 1963, 1: 998, 0: 895, 6: 658, 9: 488, 13: 398, 2: 356, 4: 272, 8: 101, 12: 87, 3: 77, 11: 71, 7: 68, 10: 3})
Interval for A3
Counter({9: 1555, 1: 951, 10: 864, 3: 541, 11: 471, 8: 440, 4: 328, 7: 300, 12: 256, 5: 229, 2: 199, 0: 132, 6: 73, 13: 53, 14: 43})
Interval for A4
Counter({4: 2157, 2: 1186, 6: 705, 8: 629, 7: 446, 13: 400, 5: 282, 3: 179, 1: 140, 0: 109, 12: 100, 9: 54, 11: 29, 10: 19})
Interval for A5
Counter({5: 2343, 3: 884, 1: 762, 6: 746, 8: 435, 0: 261, 12: 259, 2: 242, 11: 193, 7: 136, 4: 71, 9: 66, 10: 37})
Interval for A6
Counter({4: 1755, 3: 1111, 1: 986, 0: 869, 7: 436, 11: 400, 2: 228, 5: 201, 6: 200, 10: 128, 8: 78, 9: 43})
Interval for A7
Counter({10: 1764, 2: 892, 12: 686, 11: 642, 1: 627, 5: 307, 3: 267, 0: 254, 9: 227, 8: 220, 7: 212, 4: 147, 13: 130, 6: 60})
Interval for A8


In [9]:
print('Frequency for iris with 15 Intervals')
for i in num_list15:
    print('Interval for ' + i)
    print(Counter(satimage15[i]))

Frequency for iris with 15 Intervals
Interval for A1
Counter({3: 1324, 8: 921, 1: 779, 11: 622, 6: 434, 10: 392, 7: 372, 0: 269, 4: 250, 17: 247, 5: 156, 16: 138, 2: 138, 13: 114, 15: 99, 9: 73, 14: 62, 12: 45})
Interval for A2
Counter({7: 1522, 0: 1145, 1: 915, 4: 441, 11: 413, 6: 378, 8: 280, 2: 222, 20: 219, 5: 200, 19: 179, 3: 77, 13: 75, 9: 72, 18: 69, 12: 68, 10: 68, 15: 40, 16: 31, 17: 18, 14: 3})
Interval for A3
Counter({16: 1040, 1: 723, 17: 515, 3: 482, 15: 466, 18: 398, 19: 241, 20: 230, 4: 228, 14: 226, 12: 217, 13: 214, 7: 211, 0: 208, 8: 178, 21: 140, 2: 123, 5: 117, 22: 116, 6: 83, 10: 73, 9: 59, 24: 53, 11: 51, 25: 26, 23: 17})
Interval for A4
Counter({7: 1670, 1: 687, 9: 499, 5: 487, 12: 371, 13: 363, 14: 344, 4: 342, 8: 285, 23: 208, 22: 192, 0: 184, 11: 145, 2: 140, 6: 137, 3: 125, 21: 95, 10: 54, 15: 34, 16: 20, 20: 16, 17: 14, 19: 13, 18: 10})
Interval for A5
Counter({7: 1751, 3: 597, 6: 592, 0: 514, 1: 509, 8: 386, 10: 360, 5: 287, 11: 276, 17: 259, 2: 242, 12: 15

In [10]:
satimage6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 37 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   A1      6435 non-null   int64
 1   A2      6435 non-null   int64
 2   A3      6435 non-null   int64
 3   A4      6435 non-null   int64
 4   A5      6435 non-null   int64
 5   A6      6435 non-null   int64
 6   A7      6435 non-null   int64
 7   A8      6435 non-null   int64
 8   A9      6435 non-null   int64
 9   A10     6435 non-null   int64
 10  A11     6435 non-null   int64
 11  A12     6435 non-null   int64
 12  A13     6435 non-null   int64
 13  A14     6435 non-null   int64
 14  A15     6435 non-null   int64
 15  A16     6435 non-null   int64
 16  A17     6435 non-null   int64
 17  A18     6435 non-null   int64
 18  A19     6435 non-null   int64
 19  A20     6435 non-null   int64
 20  A21     6435 non-null   int64
 21  A22     6435 non-null   int64
 22  A23     6435 non-null   int64
 23  A24     6435 

In [11]:
train= satimage6.head(4435)
test= satimage6.tail(2000)
train
test

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A28,A29,A30,A31,A32,A33,A34,A35,A36,class
4435,0,0,2,2,0,0,3,2,0,0,...,1,0,0,3,2,0,0,3,2,2
4436,2,2,1,1,2,2,1,0,2,2,...,2,1,1,3,2,0,0,3,2,5
4437,2,1,2,2,2,2,3,2,2,2,...,1,2,2,3,2,2,2,3,2,4
4438,2,2,0,1,2,2,2,0,2,2,...,2,2,2,1,1,2,2,2,0,3
4439,1,2,2,2,2,1,2,2,2,1,...,1,0,0,3,2,0,1,3,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6430,2,2,0,0,2,2,0,0,2,2,...,1,2,2,2,1,2,2,2,0,5
6431,1,1,2,2,2,2,2,2,2,2,...,1,1,2,2,2,0,1,3,2,3
6432,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,1
6433,2,2,0,0,2,2,0,0,2,2,...,0,2,2,0,0,2,2,0,1,4


# 1. Categorical Naive Bayes

## 1.1 small Intervals from DT

## 1.1 debug version

In [12]:
train= satimage6.head(4435)
test= satimage6.tail(2000)
X_train = train.drop('class', axis=1)
y_train = train['class']
X_test = test.drop('class', axis=1)
y_test = test['class']

In [13]:
import time
start = time.time() # For measuring time execution

cnb = CategoricalNB()
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

end = time.time()
print(end - start) # Total time execution for this sample


0.02794337272644043


In [14]:
# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

CategoricalNB
Misclassified samples: 526
Accuracy: 0.74
Recall score :  0.737
Precision score :  0.737
F1 score :  0.737
Classification report:
              precision    recall  f1-score   support

           0       0.88      0.78      0.83       489
           1       0.98      0.91      0.95       202
           2       0.88      0.92      0.90       424
           3       0.42      0.59      0.50       214
           4       0.35      0.27      0.30       227
           5       0.71      0.74      0.72       444

    accuracy                           0.74      2000
   macro avg       0.71      0.70      0.70      2000
weighted avg       0.74      0.74      0.74      2000



In [15]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.263
Average bias: 0.263
Average variance: 0.018
Sklearn 0-1 loss: 0.263


## 1.1 script version

In [16]:
# make test & train split
train= satimage6.head(4435)
test= satimage6.tail(2000)
X_train = train.drop('class', axis=1)
y_train = train['class']
X_test = test.drop('class', axis=1)
y_test = test['class']

import time
start = time.time() # For measuring time execution

cnb = CategoricalNB()
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 526
Accuracy: 0.74
Recall score :  0.737
Precision score :  0.737
F1 score :  0.737
Classification report:
              precision    recall  f1-score   support

           0       0.88      0.78      0.83       489
           1       0.98      0.91      0.95       202
           2       0.88      0.92      0.90       424
           3       0.42      0.59      0.50       214
           4       0.35      0.27      0.30       227
           5       0.71      0.74      0.72       444

    accuracy                           0.74      2000
   macro avg       0.71      0.70      0.70      2000
weighted avg       0.74      0.74      0.74      2000

Computation time:
0.029172420501708984


In [17]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.263
Average bias: 0.263
Average variance: 0.018
Sklearn 0-1 loss: 0.263


## 1.2 medium Intervals from DT

In [18]:
# make test & train split
train= satimage8.head(4435)
test= satimage8.tail(2000)
X_train = train.drop('class', axis=1)
y_train = train['class']
X_test = test.drop('class', axis=1)
y_test = test['class']

import time
start = time.time() # For measuring time execution

cnb = CategoricalNB()
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 352
Accuracy: 0.82
Recall score :  0.824
Precision score :  0.824
F1 score :  0.824
Classification report:
              precision    recall  f1-score   support

           0       0.91      0.84      0.87       489
           1       0.98      0.91      0.95       202
           2       0.89      0.93      0.91       424
           3       0.54      0.65      0.59       214
           4       0.65      0.76      0.70       227
           5       0.89      0.79      0.83       444

    accuracy                           0.82      2000
   macro avg       0.81      0.81      0.81      2000
weighted avg       0.84      0.82      0.83      2000

Computation time:
0.0462651252746582


In [19]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.176
Average bias: 0.176
Average variance: 0.012
Sklearn 0-1 loss: 0.176


## 1.3 large Intervals from DT

In [20]:
train= satimage10.head(4435)
test= satimage10.tail(2000)
X_train = train.drop('class', axis=1)
y_train = train['class']
X_test = test.drop('class', axis=1)
y_test = test['class']

import time
start = time.time() # For measuring time execution

cnb = CategoricalNB()
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 353
Accuracy: 0.82
Recall score :  0.8235
Precision score :  0.8235
F1 score :  0.8235
Classification report:
              precision    recall  f1-score   support

           0       0.93      0.82      0.87       489
           1       0.98      0.91      0.95       202
           2       0.88      0.92      0.90       424
           3       0.55      0.70      0.62       214
           4       0.65      0.77      0.70       227
           5       0.88      0.78      0.83       444

    accuracy                           0.82      2000
   macro avg       0.81      0.82      0.81      2000
weighted avg       0.84      0.82      0.83      2000

Computation time:
0.03098583221435547


In [21]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.176
Average bias: 0.176
Average variance: 0.012
Sklearn 0-1 loss: 0.176


## 1.4 extra large Intervals from DT

In [22]:
train= satimage15.head(4435)
test= satimage15.tail(2000)
X_train = train.drop('class', axis=1)
y_train = train['class']
X_test = test.drop('class', axis=1)
y_test = test['class']

import time
start = time.time() # For measuring time execution

cnb = CategoricalNB()
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

end = time.time()
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 347
Accuracy: 0.83
Recall score :  0.8265
Precision score :  0.8265
F1 score :  0.8265
Classification report:
              precision    recall  f1-score   support

           0       0.93      0.83      0.88       489
           1       0.99      0.92      0.95       202
           2       0.88      0.93      0.90       424
           3       0.55      0.70      0.61       214
           4       0.67      0.76      0.71       227
           5       0.88      0.78      0.83       444

    accuracy                           0.83      2000
   macro avg       0.81      0.82      0.81      2000
weighted avg       0.84      0.83      0.83      2000

0.02714395523071289


In [23]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.173
Average bias: 0.172
Average variance: 0.015
Sklearn 0-1 loss: 0.173


# 2. Decision Tree ID3

## 2.1 small Intervals from DT

### Debug version

In [24]:
train= satimage6.head(4435)
test= satimage6.tail(2000)
X_train = train.drop('class', axis=1)
y_train = train['class']
X_test = test.drop('class', axis=1)
y_test = test['class']

In [25]:
import time
start = time.time() # For measuring time execution

estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
#tree = export_graphviz(estimator.tree_, 'tree.dot', y)

y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

Computation time:
1.148970603942871


In [26]:
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))

Accuracy: 0.75
Recall score :  0.752
Precision score :  0.752
F1 score :  0.752
Classification report:
              precision    recall  f1-score   support

           0       0.84      0.86      0.85       489
           1       0.95      0.94      0.94       202
           2       0.87      0.89      0.88       424
           3       0.48      0.45      0.47       214
           4       0.44      0.38      0.41       227
           5       0.71      0.75      0.73       444

    accuracy                           0.75      2000
   macro avg       0.72      0.71      0.71      2000
weighted avg       0.75      0.75      0.75      2000



In [27]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

KeyboardInterrupt: 

### script version

In [ ]:
#make splits
train= satimage6.head(4435)
test= satimage6.tail(2000)
X_train = train.drop('class', axis=1)
y_train = train['class']
X_test = test.drop('class', axis=1)
y_test = test['class']
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
#tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))
#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

In [ ]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

## 2.2 medium Intervals from DT

In [ ]:
#make splits
train= satimage8.head(4435)
test= satimage8.tail(2000)
X_train = train.drop('class', axis=1)
y_train = train['class']
X_test = test.drop('class', axis=1)
y_test = test['class']
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train)
#tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))
#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

In [ ]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

## 2.3 large Intervals from DT

In [ ]:
#make splits
train= satimage10.head(4435)
test= satimage10.tail(2000)
X_train = train.drop('class', axis=1)
y_train = train['class']
X_test = test.drop('class', axis=1)
y_test = test['class']
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
#tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))
#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

In [ ]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

## 2.4 extra large Intervals from DT

In [ ]:
#make splits
train= satimage15.head(4435)
test= satimage15.tail(2000)
X_train = train.drop('class', axis=1)
y_train = train['class']
X_test = test.drop('class', axis=1)
y_test = test['class']
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
#tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))
#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

In [ ]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

# 3. KNN classification

In [ ]:
#read in data from ChiMerge discretization
satimage6 = pd.read_csv('ChiM_discretized_6Intervals_satimage.csv')
satimage8 = pd.read_csv('ChiM_discretized_8Intervals_satimage.csv')
satimage10 = pd.read_csv('ChiM_discretized_10Intervals_satimage.csv')
satimage15 = pd.read_csv('ChiM_discretized_15Intervals_satimage.csv')

In [ ]:
# # Complete code for data preperation
# # Read data
# #df_ewd1 = satimage6
# disc = 'EWD'
# k = 4

# # df_ewd1.info()
# # data = df_ewd1.values
# # data.shape

# # train= satimage6.head(4435)
# # test= satimage6.tail(2000)

# train= satimage6.head(3000)
# test= satimage6.tail(1000)

# data_train = train.values
# data_train.shape
# data_test = test.values
# data_test.shape
# features = satimage6.drop('class', axis = 1).columns


# # separate the data into X and y
# x_train = data_train[:, : len(features)]
# y_train = data_train[:,-1]
# x_test = data_test[:, : len(features)]
# y_test = data_test[:,-1]

# #print(X.shape, Y.shape)

# # Split train test
# #x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state = 30) 

# # Check representation of class
# #print('Class representation - original: ', Counter(Y)) 
# print('Class representation - training data: ', Counter(y_train)) 
# print('Class representation - testing data: ', Counter(y_test)) 

In [ ]:
# # Knn-VDM complete code
# import time
# start = time.time() # For measuring time execution

# # specific the continuous columns index if any
# vdm = ValueDifferenceMetric(x_train, y_train, continuous = None)
# vdm.fit()
# # Knn model, n_neigbour = 3, metrics = vdm
# knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# # Fit model
# knn_vdm.fit(x_train, y_train)
# # Testing
# y_pred_knn = knn_vdm.predict(x_test)
# knn_vdm.classes_
# print(classification_report(y_test, y_pred_knn))

# end = time.time()
# print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

In [ ]:
# avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
#         knn_vdm, x_train, y_train, x_test, y_test, 
#         loss='0-1_loss',
#         random_seed=123)

# print('Average expected loss: %.3f' % avg_expected_loss)
# print('Average bias: %.3f' % avg_bias)
# print('Average variance: %.3f' % avg_var)
# print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

## 3.1 KNN with DT small intervals

In [72]:
# Complete code for data preperation
# Read data
#df_ewd1 = satimage6
disc = 'EWD'
k = 4

# df_ewd1.info()
# data = df_ewd1.values
# data.shape

# train= satimage6.head(4435)
# test= satimage6.tail(2000)

train= satimage6.head(500)
test= satimage6.tail(100)

data_train = train.values
data_train.shape
data_test = test.values
data_test.shape
features = satimage6.drop('class', axis = 1).columns


# separate the data into X and y
x_train = data_train[:, : len(features)]
y_train = data_train[:,-1]
x_test = data_test[:, : len(features)]
y_test = data_test[:,-1]

#print(X.shape, Y.shape)

# Split train test
#x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state = 30) 

# Check representation of class
#print('Class representation - original: ', Counter(Y)) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

Class representation - training data:  Counter({5: 137, 2: 111, 0: 108, 1: 53, 3: 46, 4: 45})
Class representation - testing data:  Counter({2: 25, 0: 22, 1: 16, 5: 16, 3: 13, 4: 8})


In [73]:
for i in satimage6.columns:
    #print('Interval numbers for '+i)
    print(len(Counter(train[i])))

4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
6


In [74]:
for i in train.columns:
    #print('Interval numbers for '+i)
    print(len(Counter(train[i])))

4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
6


In [75]:
for i in test.columns:
    #print('Interval numbers for '+i)
    print(len(Counter(train[i])))

4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
6


In [76]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(x_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(x_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(x_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

              precision    recall  f1-score   support

           0       0.78      0.95      0.86        22
           1       1.00      0.69      0.81        16
           2       0.80      0.96      0.87        25
           3       0.60      0.23      0.33        13
           4       1.00      0.38      0.55         8
           5       0.62      0.94      0.75        16

    accuracy                           0.77       100
   macro avg       0.80      0.69      0.70       100
weighted avg       0.79      0.77      0.74       100

Time for training model Knn-VDM: 88.16726541519165.


In [31]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        knn_vdm, x_train, y_train, x_test, y_test, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

KeyboardInterrupt: 

## 3.2 KNN with DT medium intervals

In [77]:
# Complete code for data preperation
# Read data
#df_ewd1 = satimage6
disc = 'EWD'
k = 4

# df_ewd1.info()
# data = df_ewd1.values
# data.shape

# train= satimage6.head(4435)
# test= satimage6.tail(2000)

train= satimage6.head(500)
test= satimage6.tail(100)

data_train = train.values
data_train.shape
data_test = test.values
data_test.shape
features = satimage6.drop('class', axis = 1).columns


# separate the data into X and y
x_train = data_train[:, : len(features)]
y_train = data_train[:,-1]
x_test = data_test[:, : len(features)]
y_test = data_test[:,-1]

#print(X.shape, Y.shape)

# Split train test
#x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state = 30) 

# Check representation of class
#print('Class representation - original: ', Counter(Y)) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

Class representation - training data:  Counter({5: 137, 2: 111, 0: 108, 1: 53, 3: 46, 4: 45})
Class representation - testing data:  Counter({2: 25, 0: 22, 1: 16, 5: 16, 3: 13, 4: 8})


In [78]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(x_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(x_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(x_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

              precision    recall  f1-score   support

           0       0.78      0.95      0.86        22
           1       1.00      0.69      0.81        16
           2       0.80      0.96      0.87        25
           3       0.60      0.23      0.33        13
           4       1.00      0.38      0.55         8
           5       0.62      0.94      0.75        16

    accuracy                           0.77       100
   macro avg       0.80      0.69      0.70       100
weighted avg       0.79      0.77      0.74       100

Time for training model Knn-VDM: 92.01916193962097.


In [ ]:
# avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
#         knn_vdm, X_train.values, y_train.values, X_test.values, y_test.values, 
#         loss='0-1_loss',
#         random_seed=123)

# print('Average expected loss: %.3f' % avg_expected_loss)
# print('Average bias: %.3f' % avg_bias)
# print('Average variance: %.3f' % avg_var)
# print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

## 3.3 KNN with DT large intervals

In [84]:
# Complete code for data preperation
# Read data
#df_ewd1 = satimage6
disc = 'EWD'
k = 4

# df_ewd1.info()
# data = df_ewd1.values
# data.shape

# train= satimage6.head(4435)
# test= satimage6.tail(2000)

train= satimage10.head(1000)
test= satimage10.tail(500)

data_train = train.values
data_train.shape
data_test = test.values
data_test.shape
features = satimage6.drop('class', axis = 1).columns


# separate the data into X and y
x_train = data_train[:, : len(features)]
y_train = data_train[:,-1]
x_test = data_test[:, : len(features)]
y_test = data_test[:,-1]

#print(X.shape, Y.shape)

# Split train test
#x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state = 30) 

# Check representation of class
#print('Class representation - original: ', Counter(Y)) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

Class representation - training data:  Counter({5: 260, 2: 221, 0: 214, 1: 109, 4: 106, 3: 90})
Class representation - testing data:  Counter({0: 132, 2: 115, 5: 112, 1: 49, 4: 46, 3: 46})


In [85]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(x_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(x_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(x_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

              precision    recall  f1-score   support

           0       0.92      0.98      0.95       132
           1       0.96      0.96      0.96        49
           2       0.84      0.90      0.87       115
           3       0.51      0.48      0.49        46
           4       0.76      0.57      0.65        46
           5       0.83      0.81      0.82       112

    accuracy                           0.84       500
   macro avg       0.80      0.78      0.79       500
weighted avg       0.83      0.84      0.83       500

Time for training model Knn-VDM: 948.4903318881989.


In [ ]:
# avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
#         knn_vdm, X_train.values, y_train.values, X_test.values, y_test.values, 
#         loss='0-1_loss',
#         random_seed=123)

# print('Average expected loss: %.3f' % avg_expected_loss)
# print('Average bias: %.3f' % avg_bias)
# print('Average variance: %.3f' % avg_var)
# print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

## 3.4 KNN with DT extra large intervals

In [86]:
# Complete code for data preperation
# Read data
#df_ewd1 = satimage6
disc = 'EWD'
k = 4

# df_ewd1.info()
# data = df_ewd1.values
# data.shape

# train= satimage6.head(4435)
# test= satimage6.tail(2000)

train= satimage10.head(1000)
test= satimage10.tail(500)

data_train = train.values
data_train.shape
data_test = test.values
data_test.shape
features = satimage6.drop('class', axis = 1).columns


# separate the data into X and y
x_train = data_train[:, : len(features)]
y_train = data_train[:,-1]
x_test = data_test[:, : len(features)]
y_test = data_test[:,-1]

#print(X.shape, Y.shape)

# Split train test
#x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state = 30) 

# Check representation of class
#print('Class representation - original: ', Counter(Y)) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

Class representation - training data:  Counter({5: 260, 2: 221, 0: 214, 1: 109, 4: 106, 3: 90})
Class representation - testing data:  Counter({0: 132, 2: 115, 5: 112, 1: 49, 4: 46, 3: 46})


In [87]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(x_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(x_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(x_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

              precision    recall  f1-score   support

           0       0.92      0.98      0.95       132
           1       0.96      0.96      0.96        49
           2       0.84      0.90      0.87       115
           3       0.51      0.48      0.49        46
           4       0.76      0.57      0.65        46
           5       0.83      0.81      0.82       112

    accuracy                           0.84       500
   macro avg       0.80      0.78      0.79       500
weighted avg       0.83      0.84      0.83       500

Time for training model Knn-VDM: 1114.753978252411.


In [1]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        knn_vdm, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

NameError: name 'bias_variance_decomp' is not defined